In [ ]:
import csv
import json

from buganize import Buganize

# Exporting Data

This notebook demonstrates exporting issue data to CSV and JSON using the standard library.

## Fetch Some Data

In [ ]:
async with Buganize() as client:
    result = await client.search("status:fixed type:vulnerability", page_size=50)

print(f"Fetched {len(result.issues)} issues")

## Build Row Dicts

Convert issues to a list of dicts for export. All fields are available directly on the `Issue` object.

In [ ]:
rows = []
for issue in result.issues:
    rows.append({
        "id": issue.id,
        "status": issue.status.name,
        "priority": issue.priority.name,
        "title": issue.title,
        "owner": issue.owner,
        "reporter": issue.reporter,
        "type": issue.issue_type.name if issue.issue_type else None,
        "os": issue.os,
        "cve": issue.cve,
        "milestone": issue.milestone,
        "url": issue.url,
    })

print(f"{len(rows)} rows ready for export")

## Export to CSV

In [ ]:
with open("issues.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=rows[0].keys())
    writer.writeheader()
    writer.writerows(rows)

print("Saved to issues.csv")

## Export to JSON

In [ ]:
with open("issues.json", "w") as f:
    json.dump(rows, f, indent=4, default=str)

print("Saved to issues.json")

## Filtering and Aggregating

Since issues are plain Python objects, you can filter and aggregate with list comprehensions and `collections.Counter`.

In [ ]:
from collections import Counter

# Filter by priority
p0_issues = [i for i in result.issues if i.priority.name == "P0"]
print(f"P0 issues: {len(p0_issues)}")
for issue in p0_issues[:5]:
    print(f"  #{issue.id} [{issue.status.name}] {issue.title}")

## Count by Status

In [ ]:
status_counts = Counter(i.status.name for i in result.issues)
for status, count in status_counts.most_common():
    print(f"  {status}: {count}")

## Count by Priority

In [ ]:
priority_counts = Counter(i.priority.name for i in result.issues)
for priority, count in sorted(priority_counts.items()):
    print(f"  {priority}: {count}")